# setPath

In [1]:
from sys import path
path.append(r"../")

dataDir='../Data/Causal/'
storage_path='../Data/'

# dataGenerator

In [2]:
from utils import Syn_Generator, IHDP_Generator, Twins_Generator

Syn_244 = Syn_Generator(n=1000, ate=0,sc=1,sh=0,one=1,depX=0.05,depU=0.05,VX=1,mV=1,mX=3,mU=3,init_seed=7,seed_coef=10,details=1,storage_path=storage_path)
Syn_244.run(n=1000, num_reps=10)

# IHDP_242 = IHDP_Generator(mV=2, mX=4, mU=2,details=1,dataDir=dataDir, storage_path=storage_path)
# IHDP_242.run(100)

# IHDP_260 = IHDP_Generator(mV=2, mX=6, mU=0,details=1,dataDir=dataDir, storage_path=storage_path)
# IHDP_260.run(100)

# Twins_553 = Twins_Generator(sc=1, sh=-2, mV=5, mX=5, mU=3, details=1,dataDir=dataDir, storage_path=storage_path)
# Twins_553.run(10)

# Twins_580 = Twins_Generator(sc=1, sh=-2, mV=5, mX=8, mU=0, details=1,dataDir=dataDir, storage_path=storage_path)
# Twins_580.run(10)

##############################
The data path is: ../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3/
The ATE:
------------------------------
ate: 1
------------------------------
Next, run dataGenerator: 
Run 0/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//0/
Run 1/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//1/
Run 2/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//2/
Run 3/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//3/
Run 4/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//4/
Run 5/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//5/
Run 6/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//6/
Run 7/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//7/
Run 8/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//8/
Run 9/10. 
../Data//data/Syn_1_0_1_0.05_0.05_1/1_3_3//9/
------------------------------


In [3]:
# Datasets = [Syn_244, IHDP_242, IHDP_260, Twins_553, Twins_580]
Datasets = [Syn_244]

# Pretrain: Stage1

In [4]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.Regression import run as run_Reg

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--mV',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=8,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=8,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About Regression_t
    argparser.add_argument('--regt_batch_size',default=50,type=int,help='The size of one batch')
    argparser.add_argument('--regt_lr',default=0.05,type=float,help='The learning rate')
    argparser.add_argument('--regt_num_epoch',default=3,type=int,help='The num of total epoch')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")

In [6]:
# run Syn_244_xx
for mode in ['xx','v','x']:
    data = Datasets[0]
    which_benchmark = data.which_benchmark
    which_dataset = data.which_dataset
    args.num_reps = data.num_reps
    args.mV = data.mV
    args.mX = data.mX
    args.mU = data.mU
    args.mode = mode

    resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
    dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
    os.makedirs(os.path.dirname(resultDir), exist_ok=True)
    logfile = f'{resultDir}/log.txt'

    if args.rewrite_log:
        f = open(logfile,'w')
        f.close()

    for exp in range(args.num_reps):
        train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
        val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
        test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

        train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
        val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
        test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

        train,val,test = run_Reg(exp, args, dataDir, resultDir, train, val, test, device)

Exp 0 :this is the 0/3 epochs.
The train accuracy: 73.50 %
The test  accuracy: 71.50 %
Exp 0 :this is the 1/3 epochs.
The train accuracy: 74.80 %
The test  accuracy: 71.80 %
Exp 0 :this is the 2/3 epochs.
The train accuracy: 75.40 %
The test  accuracy: 71.90 %
Exp 1 :this is the 0/3 epochs.
The train accuracy: 74.00 %
The test  accuracy: 70.00 %
Exp 1 :this is the 1/3 epochs.
The train accuracy: 74.40 %
The test  accuracy: 70.80 %
Exp 1 :this is the 2/3 epochs.
The train accuracy: 75.10 %
The test  accuracy: 71.40 %
Exp 2 :this is the 0/3 epochs.
The train accuracy: 73.40 %
The test  accuracy: 72.60 %
Exp 2 :this is the 1/3 epochs.
The train accuracy: 74.30 %
The test  accuracy: 72.40 %
Exp 2 :this is the 2/3 epochs.
The train accuracy: 75.30 %
The test  accuracy: 72.50 %
Exp 3 :this is the 0/3 epochs.
The train accuracy: 72.40 %
The test  accuracy: 72.20 %
Exp 3 :this is the 1/3 epochs.
The train accuracy: 72.80 %
The test  accuracy: 72.90 %
Exp 3 :this is the 2/3 epochs.
The train ac

In [26]:
pd.set_option("display.max_columns",30)
train.to_pandas()

,u0,u1,u2,u3,u4,u5,u6,u7,x0,x1,x2,x3,x4,x5,x6,x7,v0,p0,c0,c1,c2,c3,c4,c5,c6,c7,c8
0,-0.305333,-0.521681,-0.293908,0.046485,2.033343,0.804053,0.093124,1.180217,0.651089,-0.056812,0.541166,1.214146,-0.732179,1.932880,1.554492,0.780007,-1.471120,0.999652,-1.471120,0.651089,-0.056812,0.541166,1.214146,-0.732179,1.932880,1.554492,0.780007
1,-0.740630,0.176070,0.730925,0.139021,-1.298275,2.339401,0.873319,1.459062,-1.126615,1.446033,0.576858,-0.368737,0.382381,0.391809,-0.517528,0.728655,-0.024585,0.994619,-0.024585,-1.126615,1.446033,0.576858,-0.368737,0.382381,0.391809,-0.517528,0.728655
2,0.321667,0.997888,0.170632,0.406895,-0.989658,-1.645646,-1.197359,-0.314624,1.178024,1.717708,-0.592439,0.248040,0.214269,0.738108,0.471418,-0.222142,0.790054,0.919345,0.790054,1.178024,1.717708,-0.592439,0.248040,0.214269,0.738108,0.471418,-0.222142
3,0.109053,-0.460379,0.664466,0.235189,0.847514,1.794737,0.257834,-0.057077,0.530102,2.119603,-0.392324,-0.460401,0.163043,1.278547,-1.183967,-0.295557,1.396883,0.997243,1.396883,0.530102,2.119603,-0.392324,-0.460401,0.163043,1.278547,-1.183967,-0.295557
4,-1.465920,-1.512816,0.880548,-0.488804,-0.876977,-0.646261,-3.497859,0.546869,-0.201188,0.834670,-0.773912,0.337097,0.552986,-0.828824,-0.299133,0.176654,-0.734452,0.000812,-0.734452,-0.201188,0.834670,-0.773912,0.337097,0.552986,-0.828824,-0.299133,0.176654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.683689,-0.356409,0.422457,0.603016,-2.338384,-0.117767,0.529620,1.303065,0.387299,-1.086090,-0.434954,0.164279,1.323972,0.973463,-1.372242,0.081257,0.677665,0.736670,0.677665,0.387299,-1.086090,-0.434954,0.164279,1.323972,0.973463,-1.372242,0.081257
996,2.215619,0.443788,0.431137,-0.288254,1.060930,0.058011,2.367771,0.518598,-0.905221,-1.312704,-1.095501,0.312906,1.700087,0.242754,-1.470882,-1.023538,0.111755,0.959079,0.111755,-0.905221,-1.312704,-1.095501,0.312906,1.700087,0.242754,-1.470882,-1.023538
997,-0.452172,0.715247,0.076994,0.037955,-0.350218,-0.135859,-0.698173,-0.183249,-1.891801,-1.455445,-0.329972,1.747876,0.694354,0.357048,-1.086130,-1.392791,0.568286,0.004401,0.568286,-1.891801,-1.455445,-0.329972,1.747876,0.694354,0.357048,-1.086130,-1.392791
998,1.221548,-0.188317,-0.272450,-1.962663,-0.437557,0.617006,1.225644,0.362263,-0.393642,-0.824572,1.217129,-0.252430,-0.207252,-0.056322,0.128566,-0.345122,0.472186,0.412402,0.472186,-0.393642,-0.824572,1.217129,-0.252430,-0.207252,-0.056322,0.128566,-0.345122


In [5]:
# run vx
for data in Datasets:
    which_benchmark = data.which_benchmark
    which_dataset = data.which_dataset
    args.num_reps = data.num_reps
    args.mV = data.mV
    args.mX = data.mX
    args.mU = data.mU
    args.mode = 'vx'

    resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
    dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
    os.makedirs(os.path.dirname(resultDir), exist_ok=True)
    logfile = f'{resultDir}/log.txt'

    if args.rewrite_log:
        f = open(logfile,'w')
        f.close()

    for exp in range(args.num_reps):
        train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
        val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
        test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

        train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
        val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
        test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

        train,val,test = run_Reg(exp, args, dataDir, resultDir, train, val, test, device)

Exp 0 :this is the 0/3 epochs.
The train accuracy: 73.50 %
The test  accuracy: 71.50 %
Exp 0 :this is the 1/3 epochs.
The train accuracy: 74.80 %
The test  accuracy: 71.80 %
Exp 0 :this is the 2/3 epochs.
The train accuracy: 75.40 %
The test  accuracy: 71.90 %
Exp 1 :this is the 0/3 epochs.
The train accuracy: 74.00 %
The test  accuracy: 70.00 %
Exp 1 :this is the 1/3 epochs.
The train accuracy: 74.40 %
The test  accuracy: 70.80 %
Exp 1 :this is the 2/3 epochs.
The train accuracy: 75.10 %
The test  accuracy: 71.40 %
Exp 2 :this is the 0/3 epochs.
The train accuracy: 73.40 %
The test  accuracy: 72.60 %
Exp 2 :this is the 1/3 epochs.
The train accuracy: 74.30 %
The test  accuracy: 72.40 %
Exp 2 :this is the 2/3 epochs.
The train accuracy: 75.30 %
The test  accuracy: 72.50 %
Exp 3 :this is the 0/3 epochs.
The train accuracy: 72.40 %
The test  accuracy: 72.20 %
Exp 3 :this is the 1/3 epochs.
The train accuracy: 72.80 %
The test  accuracy: 72.90 %
Exp 3 :this is the 2/3 epochs.
The train ac

In [ ]:
args.regt_batch_size